In [1]:
from pycaret.classification import *
import xgboost
import pandas as pd

# INITIALIZE DATAFRAMES
df_train = pd.read_csv("./data/train.csv")
df_test = pd.read_csv("./data/test.csv")

In [2]:
df_train

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8688,9276_01,Europa,False,A/98/P,55 Cancri e,41.0,True,0.0,6819.0,0.0,1643.0,74.0,Gravior Noxnuther,False
8689,9278_01,Earth,True,G/1499/S,PSO J318.5-22,18.0,False,0.0,0.0,0.0,0.0,0.0,Kurta Mondalley,False
8690,9279_01,Earth,False,G/1500/S,TRAPPIST-1e,26.0,False,0.0,0.0,1872.0,1.0,0.0,Fayey Connon,True
8691,9280_01,Europa,False,E/608/S,55 Cancri e,32.0,False,0.0,1049.0,0.0,353.0,3235.0,Celeon Hontichre,False


In [5]:
df_train.count()

PassengerId     8693
HomePlanet      8492
CryoSleep       8476
Cabin           8494
Destination     8511
Age             8514
VIP             8490
RoomService     8512
FoodCourt       8510
ShoppingMall    8485
Spa             8510
VRDeck          8505
Name            8493
Transported     8693
dtype: int64

In [3]:
df_train.HomePlanet.unique()

array(['Europa', 'Earth', 'Mars', nan], dtype=object)

In [6]:
df_train['HomePlanet'].value_counts(dropna=False)

Earth     4602
Europa    2131
Mars      1759
NaN        201
Name: HomePlanet, dtype: int64

In [9]:
df_train.describe()

,Age,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck
count,8514.000000,8512.000000,8510.000000,8485.000000,8510.000000,8505.000000
mean,28.827930,224.687617,458.077203,173.729169,311.138778,304.854791
std,14.489021,666.717663,1611.489240,604.696458,1136.705535,1145.717189
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,19.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,27.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,38.000000,47.000000,76.000000,27.000000,59.000000,46.000000
max,79.000000,14327.000000,29813.000000,23492.000000,22408.000000,24133.000000


In [10]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8693 entries, 0 to 8692
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   PassengerId   8693 non-null   object 
 1   HomePlanet    8492 non-null   object 
 2   CryoSleep     8476 non-null   object 
 3   Cabin         8494 non-null   object 
 4   Destination   8511 non-null   object 
 5   Age           8514 non-null   float64
 6   VIP           8490 non-null   object 
 7   RoomService   8512 non-null   float64
 8   FoodCourt     8510 non-null   float64
 9   ShoppingMall  8485 non-null   float64
 10  Spa           8510 non-null   float64
 11  VRDeck        8505 non-null   float64
 12  Name          8493 non-null   object 
 13  Transported   8693 non-null   bool   
dtypes: bool(1), float64(6), object(7)
memory usage: 891.5+ KB


In [11]:
df_train['Transported'] = df_train['Transported'].astype(int)

In [12]:
df_train.head()

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,0
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,1
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,0
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,0
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,1


In [14]:
df_train['CryoSleep'].value_counts(dropna=False)

False    5439
True     3037
NaN       217
Name: CryoSleep, dtype: int64

In [15]:
df_train['CryoSleep'] = df_train['CryoSleep'].fillna(value=0)

In [16]:
df_train['CryoSleep'].value_counts(dropna=False)

False    5656
True     3037
Name: CryoSleep, dtype: int64

In [17]:
df_train['CryoSleep'] = df_train['CryoSleep'].astype(int)

In [18]:
df_train.head()

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,0,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,0
1,0002_01,Earth,0,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,1
2,0003_01,Europa,0,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,0
3,0003_02,Europa,0,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,0
4,0004_01,Earth,0,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,1


In [19]:
df_train['VIP'].value_counts(dropna=False)

False    8291
NaN       203
True      199
Name: VIP, dtype: int64

In [20]:
df_train['VIP'] = df_train['VIP'].fillna(value=0)

In [21]:
df_train['VIP'] = df_train['VIP'].astype(int)

In [54]:
df_train.head()

,HomePlanet,CryoSleep,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Transported,Deck,Cabin_num,Side
0,Europa,0,TRAPPIST-1e,39.0,0,0.0,0.0,0.0,0.0,0.0,0,B,0,P
1,Earth,0,TRAPPIST-1e,24.0,0,109.0,9.0,25.0,549.0,44.0,1,F,0,S
2,Europa,0,TRAPPIST-1e,58.0,1,43.0,3576.0,0.0,6715.0,49.0,0,A,0,S
3,Europa,0,TRAPPIST-1e,33.0,0,0.0,1283.0,371.0,3329.0,193.0,0,A,0,S
4,Earth,0,TRAPPIST-1e,16.0,0,303.0,70.0,151.0,565.0,2.0,1,F,1,S


In [53]:
df_train = df_train.drop('Name', axis=1)

KeyError: "['Name'] not found in axis"

In [24]:
df_train.head()

,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Transported
0,Europa,0,B/0/P,TRAPPIST-1e,39.0,0,0.0,0.0,0.0,0.0,0.0,0
1,Earth,0,F/0/S,TRAPPIST-1e,24.0,0,109.0,9.0,25.0,549.0,44.0,1
2,Europa,0,A/0/S,TRAPPIST-1e,58.0,1,43.0,3576.0,0.0,6715.0,49.0,0
3,Europa,0,A/0/S,TRAPPIST-1e,33.0,0,0.0,1283.0,371.0,3329.0,193.0,0
4,Earth,0,F/1/S,TRAPPIST-1e,16.0,0,303.0,70.0,151.0,565.0,2.0,1


In [29]:
numeric_features = ['Age', 'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']
for column in numeric_features:
    print(column)
    print("mean = " + str(df_train[column].mean()))
    print("median = " + str(df_train[column].median()))
    print()

Age
mean = 28.82793046746535
median = 27.0

RoomService
mean = 224.687617481203
median = 0.0

FoodCourt
mean = 458.07720329024676
median = 0.0

ShoppingMall
mean = 173.72916912197996
median = 0.0

Spa
mean = 311.1387779083431
median = 0.0

VRDeck
mean = 304.8547912992357
median = 0.0



In [30]:
df_train['Age'] = df_train['Age'].fillna(value=27)

In [31]:
spending_columns = ['RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']
for column in spending_columns:
    df_train[column] = df_train[column].fillna(value=0)

In [32]:
df_train.isna().sum().sum()

582

In [33]:
df_train.head()

,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Transported
0,Europa,0,B/0/P,TRAPPIST-1e,39.0,0,0.0,0.0,0.0,0.0,0.0,0
1,Earth,0,F/0/S,TRAPPIST-1e,24.0,0,109.0,9.0,25.0,549.0,44.0,1
2,Europa,0,A/0/S,TRAPPIST-1e,58.0,1,43.0,3576.0,0.0,6715.0,49.0,0
3,Europa,0,A/0/S,TRAPPIST-1e,33.0,0,0.0,1283.0,371.0,3329.0,193.0,0
4,Earth,0,F/1/S,TRAPPIST-1e,16.0,0,303.0,70.0,151.0,565.0,2.0,1


In [34]:
df_train.describe()

,CryoSleep,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Transported
count,8693.000000,8693.000000,8693.000000,8693.000000,8693.000000,8693.000000,8693.000000,8693.000000,8693.000000
mean,0.349362,28.790291,0.022892,220.009318,448.434027,169.572300,304.588865,298.261820,0.503624
std,0.476796,14.341404,0.149568,660.519050,1595.790627,598.007164,1125.562559,1134.126417,0.500016
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,20.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,27.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
75%,1.000000,37.000000,0.000000,41.000000,61.000000,22.000000,53.000000,40.000000,1.000000
max,1.000000,79.000000,1.000000,14327.000000,29813.000000,23492.000000,22408.000000,24133.000000,1.000000


In [35]:
df_train[["Deck", "Cabin_num", "Side"]] = df_train["Cabin"].str.split("/", expand=True)

In [36]:
df_train.head()

,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Transported,Deck,Cabin_num,Side
0,Europa,0,B/0/P,TRAPPIST-1e,39.0,0,0.0,0.0,0.0,0.0,0.0,0,B,0,P
1,Earth,0,F/0/S,TRAPPIST-1e,24.0,0,109.0,9.0,25.0,549.0,44.0,1,F,0,S
2,Europa,0,A/0/S,TRAPPIST-1e,58.0,1,43.0,3576.0,0.0,6715.0,49.0,0,A,0,S
3,Europa,0,A/0/S,TRAPPIST-1e,33.0,0,0.0,1283.0,371.0,3329.0,193.0,0,A,0,S
4,Earth,0,F/1/S,TRAPPIST-1e,16.0,0,303.0,70.0,151.0,565.0,2.0,1,F,1,S


In [37]:
df_train = df_train.drop('Cabin', axis=1)

In [38]:
df_train.head()

,HomePlanet,CryoSleep,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Transported,Deck,Cabin_num,Side
0,Europa,0,TRAPPIST-1e,39.0,0,0.0,0.0,0.0,0.0,0.0,0,B,0,P
1,Earth,0,TRAPPIST-1e,24.0,0,109.0,9.0,25.0,549.0,44.0,1,F,0,S
2,Europa,0,TRAPPIST-1e,58.0,1,43.0,3576.0,0.0,6715.0,49.0,0,A,0,S
3,Europa,0,TRAPPIST-1e,33.0,0,0.0,1283.0,371.0,3329.0,193.0,0,A,0,S
4,Earth,0,TRAPPIST-1e,16.0,0,303.0,70.0,151.0,565.0,2.0,1,F,1,S


In [39]:
clf1 = setup(data=df_train, target="Transported",
            categorical_features=["HomePlanet", "CryoSleep", "Destination", "VIP", 'Deck', 'Cabin_num', 'Side'], 
            numeric_features=["Age", "RoomService", "FoodCourt", "ShoppingMall", "Spa", "VRDeck"])

,Description,Value
0,Session id,7918
1,Target,Transported
2,Target type,Binary
3,Original data shape,"(8693, 14)"
4,Transformed data shape,"(8693, 25)"
5,Transformed train set shape,"(6085, 25)"
6,Transformed test set shape,"(2608, 25)"
7,Ordinal features,3
8,Numeric features,6
9,Categorical features,7


In [40]:
best = clf1.compare_models()

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
gbc,Gradient Boosting Classifier,0.8002,0.8856,0.8339,0.7837,0.8077,0.6001,0.6020,0.1200
xgboost,Extreme Gradient Boosting,0.7982,0.8805,0.7951,0.8026,0.7987,0.5964,0.5966,0.1190
lightgbm,Light Gradient Boosting Machine,0.7941,0.8839,0.7971,0.7950,0.7958,0.5881,0.5884,0.0520
rf,Random Forest Classifier,0.7900,0.8699,0.7540,0.8154,0.7832,0.5802,0.5821,0.1020
ada,Ada Boost Classifier,0.7896,0.8734,0.8108,0.7805,0.7950,0.5791,0.5801,0.0650
lr,Logistic Regression,0.7857,0.8748,0.8046,0.7780,0.7908,0.5713,0.5721,0.3580
et,Extra Trees Classifier,0.7742,0.8434,0.7282,0.8056,0.7646,0.5487,0.5517,0.0950
knn,K Neighbors Classifier,0.7719,0.8442,0.7778,0.7715,0.7745,0.5437,0.5440,0.1530
ridge,Ridge Classifier,0.7679,0.0000,0.7093,0.8070,0.7547,0.5363,0.5406,0.0300
lda,Linear Discriminant Analysis,0.7678,0.8549,0.7090,0.8069,0.7545,0.5360,0.5403,0.0380


In [41]:
tuned = tune_model(best)

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.8243,0.9077,0.8632,0.8030,0.8320,0.6484,0.6502
1,0.7750,0.8678,0.8078,0.7607,0.7836,0.5498,0.5509
2,0.8062,0.8825,0.7948,0.8161,0.8053,0.6125,0.6127
3,0.8062,0.8943,0.8013,0.8119,0.8066,0.6125,0.6125
4,0.8013,0.8811,0.8013,0.8039,0.8026,0.6026,0.6026
5,0.7763,0.8759,0.7516,0.7931,0.7718,0.5528,0.5535
6,0.8240,0.9055,0.8301,0.8220,0.8260,0.6480,0.6480
7,0.8043,0.8979,0.8301,0.7913,0.8102,0.6084,0.6092
8,0.8026,0.8818,0.7908,0.8121,0.8013,0.6053,0.6055


Fitting 10 folds for each of 10 candidates, totalling 100 fits


In [42]:
plot_model(tuned, plot="feature", save=True)

'Feature Importance.png'

In [43]:
predictions = predict_model(tuned, data=df_test, raw_score=True)

KeyError: "['Deck', 'Cabin_num', 'Side'] not in index"

In [44]:
df_test[["Deck", "Cabin_num", "Side"]] = df_test["Cabin"].str.split("/", expand=True)

In [45]:
df_test.head()

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Deck,Cabin_num,Side
0,0013_01,Earth,True,G/3/S,TRAPPIST-1e,27.0,False,0.0,0.0,0.0,0.0,0.0,Nelly Carsoning,G,3,S
1,0018_01,Earth,False,F/4/S,TRAPPIST-1e,19.0,False,0.0,9.0,0.0,2823.0,0.0,Lerome Peckers,F,4,S
2,0019_01,Europa,True,C/0/S,55 Cancri e,31.0,False,0.0,0.0,0.0,0.0,0.0,Sabih Unhearfus,C,0,S
3,0021_01,Europa,False,C/1/S,TRAPPIST-1e,38.0,False,0.0,6652.0,0.0,181.0,585.0,Meratz Caltilter,C,1,S
4,0023_01,Earth,False,F/5/S,TRAPPIST-1e,20.0,False,10.0,0.0,635.0,0.0,0.0,Brence Harperez,F,5,S


In [46]:
df_test = df_test.drop('Cabin', axis=1)

In [47]:
df_test.head()

,PassengerId,HomePlanet,CryoSleep,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Deck,Cabin_num,Side
0,0013_01,Earth,True,TRAPPIST-1e,27.0,False,0.0,0.0,0.0,0.0,0.0,Nelly Carsoning,G,3,S
1,0018_01,Earth,False,TRAPPIST-1e,19.0,False,0.0,9.0,0.0,2823.0,0.0,Lerome Peckers,F,4,S
2,0019_01,Europa,True,55 Cancri e,31.0,False,0.0,0.0,0.0,0.0,0.0,Sabih Unhearfus,C,0,S
3,0021_01,Europa,False,TRAPPIST-1e,38.0,False,0.0,6652.0,0.0,181.0,585.0,Meratz Caltilter,C,1,S
4,0023_01,Earth,False,TRAPPIST-1e,20.0,False,10.0,0.0,635.0,0.0,0.0,Brence Harperez,F,5,S


In [48]:
predictions = predict_model(tuned, data=df_test, raw_score=True)

In [51]:
predictions.head()

,HomePlanet_Europa,HomePlanet_Earth,HomePlanet_Mars,CryoSleep,Destination_TRAPPIST-1e,Destination_55 Cancri e,Destination_PSO J318.5-22,Age,VIP,RoomService,...,Deck_G,Deck_C,Deck_D,Deck_A,Deck_T,Cabin_num,Side,prediction_label,prediction_score_0,prediction_score_1
0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,27.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.583333,1.0,1,0.4768,0.5232
1,0.0,1.0,0.0,0.0,1.0,0.0,0.0,19.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.250000,1.0,0,0.9726,0.0274
2,1.0,0.0,0.0,1.0,0.0,1.0,0.0,31.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.166667,1.0,1,0.0073,0.9927
3,1.0,0.0,0.0,0.0,1.0,0.0,0.0,38.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.777778,1.0,1,0.0231,0.9769
4,0.0,1.0,0.0,0.0,1.0,0.0,0.0,20.0,0.0,10.0,...,0.0,0.0,0.0,0.0,0.0,0.333333,1.0,1,0.4247,0.5753


In [55]:
predictions.head()

,HomePlanet_Europa,HomePlanet_Earth,HomePlanet_Mars,CryoSleep,Destination_TRAPPIST-1e,Destination_55 Cancri e,Destination_PSO J318.5-22,Age,VIP,RoomService,...,Deck_G,Deck_C,Deck_D,Deck_A,Deck_T,Cabin_num,Side,prediction_label,prediction_score_0,prediction_score_1
0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,27.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.583333,1.0,1,0.4768,0.5232
1,0.0,1.0,0.0,0.0,1.0,0.0,0.0,19.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.250000,1.0,0,0.9726,0.0274
2,1.0,0.0,0.0,1.0,0.0,1.0,0.0,31.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.166667,1.0,1,0.0073,0.9927
3,1.0,0.0,0.0,0.0,1.0,0.0,0.0,38.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.777778,1.0,1,0.0231,0.9769
4,0.0,1.0,0.0,0.0,1.0,0.0,0.0,20.0,0.0,10.0,...,0.0,0.0,0.0,0.0,0.0,0.333333,1.0,1,0.4247,0.5753


In [57]:
for col in predictions.columns:
    print(col)

HomePlanet_Europa
HomePlanet_Earth
HomePlanet_Mars
CryoSleep
Destination_TRAPPIST-1e
Destination_55 Cancri e
Destination_PSO J318.5-22
Age
VIP
RoomService
FoodCourt
ShoppingMall
Spa
VRDeck
Deck_B
Deck_E
Deck_F
Deck_G
Deck_C
Deck_D
Deck_A
Deck_T
Cabin_num
Side
prediction_label
prediction_score_0
prediction_score_1
